In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
from openpyxl import Workbook,load_workbook

In [3]:
import os

In [4]:
from sklearn.metrics import r2_score,mean_squared_error,precision_score,recall_score,accuracy_score,confusion_matrix

# Computing sample weights using malhanobis diatance

In [5]:
def malhanobis_dist(invcov,array):
    return float(1)/np.sqrt(np.dot(np.dot(array,invcov),np.transpose(array)))

In [6]:
os.chdir('../output_data')

In [7]:
df = pd.read_csv('train_v04.csv',header=0)

In [9]:
df_lr = pd.read_csv('train_lr.csv',header=0)

In [8]:
df.shape

(85799, 176)

In [11]:
df_lr.shape

(85799, 35)

In [9]:
ignore_cols = ['Symbol','Series','Date','Prev_Close','Open','Close','Prev_Close_min','Prev_Close_max','Sector']

# Linear Regression Inferential Model

In [10]:
x = df.drop(ignore_cols+['actual_return','lift'],axis=1)

In [11]:
invcov = np.linalg.inv(np.array(pd.DataFrame(x).cov()))

In [12]:
x_inv = np.transpose(np.array([[pd.DataFrame(x)[col]-pd.DataFrame(x)[col].mean()] for col in pd.DataFrame(x).columns]))[:,0,:]

In [13]:
sample_weights = np.array([malhanobis_dist(invcov,x_inv[i,:]) for i in range(x_inv.shape[0])])

In [14]:
os.chdir('../pickles/')

In [15]:
pickle.dump(sample_weights,open('inferential_sample_weights_wr.pickle','wb'))

In [16]:
model = LinearRegression()

In [17]:
y = np.array(df['actual_return'])

In [18]:
model.fit(x,y,sample_weight=sample_weights)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
y_pred = model.predict(x)

In [20]:
r2_train = r2_score(y,y_pred)

In [21]:
r2_train

0.0203771499278389

In [22]:
os.chdir('../Models/')

In [23]:
pickle.dump(model,open('LR_wr.pickle','wb'))

In [24]:
model_columns = df.drop(ignore_cols+['actual_return','lift'],axis=1).columns

In [27]:
lr_model_columns = df_lr.drop(ignore_cols+['actual_return','lift'],axis=1).columns

In [25]:
os.chdir('../pickles')

In [27]:
pickle.dump(model_columns,open('model_columns.pickle','wb'))

In [28]:
df['lift_predicted'] = [max(0,np.sign(x)) for x in y_pred]

In [29]:
accuracy_score(y_true=df['lift'],y_pred=df['lift_predicted'])

0.5529085420575998

In [30]:
precision_score(y_true=df['lift'],y_pred=df['lift_predicted'])

0.528298252854588

In [31]:
recall_score(y_true=df['lift'],y_pred=df['lift_predicted'])

0.47507237053715023

In [32]:
df['month'] = df['Date'].apply(lambda x : str(x)[5:7])
df['year'] = df['Date'].apply(lambda x: str(x)[0:4])

In [33]:
df['correct_prediction'] = df[['lift','lift_predicted']].apply(lambda x: 1 if int(x[0]) == int(x[1]) else 0,axis=1)

In [34]:
df['precise_prediction'] = df[['lift','lift_predicted']].apply(lambda x: 1 if int(x[0]) == 1 and int(x[1]) == 1 else 0,axis=1)

In [35]:
df_grp = df.groupby(['month','year','Symbol','Sector']).agg({'lift':['count','sum'],'lift_predicted':['sum'],
                                                          'correct_prediction':['sum'],'precise_prediction':['sum']})

In [36]:
df_grp.reset_index(inplace=True)

In [37]:
df_grp.columns = ['month','year','Symbol','Sector','trade_days','price_lift','pred_price_lift','correct_predcitions','precise_predictions']

In [38]:
df_grp

,month,year,Symbol,Sector,trade_days,price_lift,pred_price_lift,correct_predcitions,precise_predictions
0,01,2012,ADANIPORTS,Services - Shipping,17,10,6.0,9,4
1,01,2012,ASIANPAINT,Consumer Goods,17,9,9.0,11,6
2,01,2012,AXISBANK,Banking,17,12,9.0,8,6
3,01,2012,BAJAJ-AUTO,Automobile,17,11,7.0,9,5
4,01,2012,BAJAJFINSV,Financial Services,17,7,11.0,9,5
5,01,2012,BAJFINANCE,Financial Services,17,11,9.0,11,7
6,01,2012,BHARTIARTL,Telecommunication,17,10,10.0,13,8
7,01,2012,BPCL,Energy - Oil & Gas,17,10,6.0,7,3
8,01,2012,BRITANNIA,Consumer Goods,17,10,10.0,7,5
9,01,2012,CIPLA,Pharmaceuticals,17,7,7.0,11,4


In [39]:
os.chdir('../Statistics/')

In [40]:
wb = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx',engine='openpyxl')
writer.book = wb
df_grp.to_excel(writer,sheet_name='lr_price_stats_wr',index=False)
writer.save()
writer.close()

In [41]:
os.chdir('../Models/')

In [42]:
model = pickle.load(open('LR_wr.pickle','rb'))

In [43]:
equation = str(model.intercept_)+" + "

In [44]:
for col,coef in zip(model_columns,model.coef_):
    equation+=str(coef)+" x {col} + ".format(col=col)

In [45]:
equation = equation[:-2]

In [46]:
equation

'-0.013304435167282011 + -0.03769963449662852 x High_pct_change + -0.0007271745537302966 x Low_pct_change + 0.06910125332108118 x Last_pct_change + 0.033570618009735965 x VWAP_pct_change + 0.008341789751108612 x Volume_pct_change + -0.008624840407063248 x Turnover_pct_change + -0.0003101714824664133 x Trades_pct_change + 0.0005635019933060222 x Deliverable_Volume_pct_change + -0.0005849839504987681 x pct_Deliverble_pct_change + -0.992983271307938 x Open_nifty_pct_change + -0.18193288637849656 x High_nifty_pct_change + -0.11819543001872285 x Low_nifty_pct_change + 0.7900472874809981 x Close_nifty_pct_change + -0.0001433332203127774 x Volume_nifty_pct_change + 0.0006180858797546797 x Turnover_nifty_pct_change + -0.25001594698161345 x Open_auto_pct_change + 0.05215801917534409 x High_auto_pct_change + 0.000479059777956696 x Low_auto_pct_change + 0.16787015643688116 x Close_auto_pct_change + 0.0009754279423531218 x Volume_auto_pct_change + 0.0006539802515870803 x Turnover_auto_pct_change +

In [47]:
df_equation = pd.DataFrame()
df_equation['columns'] = ['intercept'] + list(model_columns)
df_equation['coef'] = [model.intercept_] + list(model.coef_)

In [48]:
os.chdir('../Statistics/')

In [49]:
wb = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx',engine='openpyxl')
writer.book = wb
df_equation.to_excel(writer,sheet_name='lr_price_model_wr',index=False)
writer.save()
writer.close()

In [50]:
model_columns

Index(['High_pct_change', 'Low_pct_change', 'Last_pct_change',
       'VWAP_pct_change', 'Volume_pct_change', 'Turnover_pct_change',
       'Trades_pct_change', 'Deliverable_Volume_pct_change',
       'pct_Deliverble_pct_change', 'Open_nifty_pct_change',
       ...
       'Sector_dum_Metals', 'Sector_dum_Information_Technology',
       'Sector_dum_Financial_Services', 'Sector_dum_Pharmaceuticals',
       'Sector_dum_Cement', 'Sector_dum_Energy___Power',
       'Sector_dum_Telecommunication', 'Sector_dum_Construction',
       'Sector_dum_Media___Entertainment', 'Sector_dum_Services___Shipping'],
      dtype='object', length=165)

In [51]:
os.chdir('../pickles/')

In [52]:
pickle.dump(model_columns,open('model_columns.pickle','wb'))

# Random Forest Inferential Model

In [26]:
os.chdir('../Models')

In [27]:
tuned_parameters={'max_features':[None,'sqrt','log2'],'min_samples_leaf':[i*.01 for i in range(1,11)]}
model = GridSearchCV(RandomForestRegressor(n_jobs=-1,n_estimators=1000,oob_score=True,random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=1000)

In [28]:
x = df_lr.drop(['actual_return'],axis=1)

In [29]:
y = np.array(df_lr['actual_return'])

In [30]:
model.fit(x,y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=-0.003465 - 4.2min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.2min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=-0.003907 - 9.8min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 14.1min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=-0.006905 - 9.4min
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 23.5min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=-0.000100 - 9.1min
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 32.6min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0

[CV]  max_features=None, min_samples_leaf=0.08, score=-0.001543 - 1.7min
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed: 138.8min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=-0.004100 - 1.5min
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed: 140.4min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=-0.001189 - 1.8min
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed: 142.1min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=-0.006106 - 1.6min
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 143.7min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.09 ........................
[CV]  max_features=None, min_samples_leaf=0.09, score=-0.003193 - 1.5min
[Parallel(n_jobs=1)]: Done 

[CV]  max_features=sqrt, min_samples_leaf=0.05, score=-0.001060 -  27.9s
[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed: 172.7min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.05 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.05, score=-0.003129 -  27.8s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed: 173.2min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=-0.001332 -  25.2s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed: 173.6min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=-0.001223 -  25.9s
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed: 174.1min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=-0.001122 -  25.5s
[Parallel(n_jobs=1)]: Done 

[CV]  max_features=log2, min_samples_leaf=0.03, score=-0.001648 -  33.7s
[Parallel(n_jobs=1)]: Done 111 out of 111 | elapsed: 190.9min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=-0.000962 -  36.1s
[Parallel(n_jobs=1)]: Done 112 out of 112 | elapsed: 191.5min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=-0.001355 -  34.7s
[Parallel(n_jobs=1)]: Done 113 out of 113 | elapsed: 192.0min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=-0.001016 -  35.8s
[Parallel(n_jobs=1)]: Done 114 out of 114 | elapsed: 192.6min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=-0.004446 -  37.0s
[Parallel(n_jobs=1)]: Done 

[CV]  max_features=log2, min_samples_leaf=0.1, score=-0.002145 -  22.4s
[Parallel(n_jobs=1)]: Done 148 out of 148 | elapsed: 207.6min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.1 .........................
[CV]  max_features=log2, min_samples_leaf=0.1, score=-0.001463 -  21.8s
[Parallel(n_jobs=1)]: Done 149 out of 149 | elapsed: 208.0min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.1 .........................
[CV]  max_features=log2, min_samples_leaf=0.1, score=-0.001717 -  21.7s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 208.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 208.3min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=1000)

In [31]:
model.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.07, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [32]:
model.best_score_

-0.001462095376883766

In [33]:
final_model = model.best_estimator_

In [34]:
final_model.fit(x,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.07, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [35]:
y_pred = final_model.predict(x)

In [36]:
r2_score(y,y_pred)

0.006947679027599629

In [37]:
pickle.dump(final_model,open('RF.pickle','wb'))

In [38]:
os.chdir('../pickles/')

In [40]:
model_columns = df.drop(['actual_return'],axis=1).columns
pickle.dump(model_columns,open('model_columns','wb'))

# Gradient Boosting Inferential Model

In [41]:
os.chdir('../Models')

In [45]:
tuned_parameters={'n_estimators':[10*i for i in range(10,51,10)]}
model = GridSearchCV(GradientBoostingRegressor(learning_rate=.1,min_samples_leaf=.01,max_depth=10,subsample=.8,
                                               random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=50)

In [46]:
model.fit(x,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=100 ................................................
[CV] ...................... n_estimators=100, score=-0.017520 -  42.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   42.2s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ...................... n_estimators=100, score=-0.016555 -  47.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.5min remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ...................... n_estimators=100, score=-0.029091 - 1.1min
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.6min remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ...................... n_estimators=100, score=-0.021524 -  45.2s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  3.4min remaining:    0.0s
[CV] n_estimators=100 ..............................

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=50)

In [49]:
model.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False)

In [50]:
model.best_score_

-0.024509673649512228

In [51]:
tuned_parameters={'max_depth':[8,10,12,15,20],'min_samples_leaf':[.01*i for i in range(1,11)],'subsample':[.8,.9,1]
                 ,'max_features':[None,'sqrt','log2']}
model = GridSearchCV(GradientBoostingRegressor(n_estimators=100,learning_rate=.1,random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=2500)

In [52]:
model.fit(x,y)

Fitting 5 folds for each of 450 candidates, totalling 2250 fits
[CV] max_depth=8, max_features=None, min_samples_leaf=0.01, subsample=0.8 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.01, subsample=0.8, score=-0.011061 -  28.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.1s remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.01, subsample=0.8 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.01, subsample=0.8, score=-0.014991 -  28.5s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   56.7s remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.01, subsample=0.8 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.01, subsample=0.8, score=-0.026731 -  27.8s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.4min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.01, subsample=0.8 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.01, subsample=0.8, score=-0.018309 -

[CV]  max_depth=8, max_features=None, min_samples_leaf=0.03, subsample=0.8, score=-0.024337 -  20.7s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed: 14.4min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.03, subsample=0.8 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.03, subsample=0.8, score=-0.012525 -  20.7s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed: 14.8min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.03, subsample=0.8 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.03, subsample=0.8, score=-0.027152 -  20.4s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed: 15.1min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.03, subsample=0.9 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.03, subsample=0.9, score=-0.017358 -  25.1s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 15.5min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_le

[CV]  max_depth=8, max_features=None, min_samples_leaf=0.05, subsample=0.9, score=-0.012899 -  17.4s
[Parallel(n_jobs=1)]: Done  66 out of  66 | elapsed: 25.3min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.05, subsample=0.9 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.05, subsample=0.9, score=-0.009737 -  17.2s
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed: 25.6min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.05, subsample=0.9 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.05, subsample=0.9, score=-0.020164 -  20.1s
[Parallel(n_jobs=1)]: Done  68 out of  68 | elapsed: 25.9min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.05, subsample=0.9 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.05, subsample=0.9, score=-0.011461 -  21.2s
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed: 26.3min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_le

[CV]  max_depth=8, max_features=None, min_samples_leaf=0.07, subsample=0.9, score=-0.007883 -  14.2s
[Parallel(n_jobs=1)]: Done  99 out of  99 | elapsed: 34.2min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.07, subsample=0.9 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.07, subsample=0.9, score=-0.017785 -  14.2s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 34.4min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.07, subsample=1 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.07, subsample=1, score=-0.008360 -  14.1s
[Parallel(n_jobs=1)]: Done 101 out of 101 | elapsed: 34.7min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.07, subsample=1 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.07, subsample=1, score=-0.005090 -  14.4s
[Parallel(n_jobs=1)]: Done 102 out of 102 | elapsed: 34.9min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.07,

[CV]  max_depth=8, max_features=None, min_samples_leaf=0.09, subsample=1, score=-0.007804 -  15.2s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed: 41.3min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.09, subsample=1 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.09, subsample=1, score=-0.013326 -  14.3s
[Parallel(n_jobs=1)]: Done 133 out of 133 | elapsed: 41.5min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.09, subsample=1 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.09, subsample=1, score=-0.006208 -  14.8s
[Parallel(n_jobs=1)]: Done 134 out of 134 | elapsed: 41.7min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.09, subsample=1 
[CV]  max_depth=8, max_features=None, min_samples_leaf=0.09, subsample=1, score=-0.011164 -  14.5s
[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed: 42.0min remaining:    0.0s
[CV] max_depth=8, max_features=None, min_samples_leaf=0.1, subsam

...

[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.02, subsample=1, score=-0.010288 -   6.1s
[Parallel(n_jobs=1)]: Done 2126 out of 2126 | elapsed: 339.3min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.02, subsample=1 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.02, subsample=1, score=-0.015981 -   6.1s
[Parallel(n_jobs=1)]: Done 2127 out of 2127 | elapsed: 339.4min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.02, subsample=1 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.02, subsample=1, score=-0.026646 -   5.9s
[Parallel(n_jobs=1)]: Done 2128 out of 2128 | elapsed: 339.5min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.02, subsample=1 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.02, subsample=1, score=-0.019008 -   5.9s
[Parallel(n_jobs=1)]: Done 2129 out of 2129 | elapsed: 339.6min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samp

[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.04, subsample=1, score=-0.020614 -   4.7s
[Parallel(n_jobs=1)]: Done 2158 out of 2158 | elapsed: 341.9min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.04, subsample=1 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.04, subsample=1, score=-0.015039 -   4.8s
[Parallel(n_jobs=1)]: Done 2159 out of 2159 | elapsed: 342.0min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.04, subsample=1 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.04, subsample=1, score=-0.026019 -   4.8s
[Parallel(n_jobs=1)]: Done 2160 out of 2160 | elapsed: 342.1min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.05, subsample=0.8 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.05, subsample=0.8, score=-0.001664 -   4.0s
[Parallel(n_jobs=1)]: Done 2161 out of 2161 | elapsed: 342.2min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_

[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.06, subsample=1, score=-0.016125 -   3.0s
[Parallel(n_jobs=1)]: Done 2190 out of 2190 | elapsed: 343.8min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.07, subsample=0.8 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.07, subsample=0.8, score=-0.002826 -   2.8s
[Parallel(n_jobs=1)]: Done 2191 out of 2191 | elapsed: 343.9min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.07, subsample=0.8 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.07, subsample=0.8, score=-0.006302 -   2.8s
[Parallel(n_jobs=1)]: Done 2192 out of 2192 | elapsed: 343.9min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.07, subsample=0.8 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.07, subsample=0.8, score=-0.008445 -   2.8s
[Parallel(n_jobs=1)]: Done 2193 out of 2193 | elapsed: 344.0min remaining:    0.0s
[CV] max_depth=20, max_features=lo

[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.09, subsample=0.8, score=-0.001326 -   2.4s
[Parallel(n_jobs=1)]: Done 2222 out of 2222 | elapsed: 345.3min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.09, subsample=0.8 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.09, subsample=0.8, score=-0.009259 -   2.4s
[Parallel(n_jobs=1)]: Done 2223 out of 2223 | elapsed: 345.3min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.09, subsample=0.8 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.09, subsample=0.8, score=-0.003532 -   2.4s
[Parallel(n_jobs=1)]: Done 2224 out of 2224 | elapsed: 345.4min remaining:    0.0s
[CV] max_depth=20, max_features=log2, min_samples_leaf=0.09, subsample=0.8 
[CV]  max_depth=20, max_features=log2, min_samples_leaf=0.09, subsample=0.8, score=-0.007250 -   2.4s
[Parallel(n_jobs=1)]: Done 2225 out of 2225 | elapsed: 345.4min remaining:    0.0s
[CV] max_depth=20, max_features=

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [8, 10, 12, 15, 20], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1], 'subsample': [0.8, 0.9, 1], 'max_features': [None, 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=2500)

In [53]:
model.best_score_

-0.0046664667980570815

In [54]:
model.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=42, subsample=0.8, verbose=0,
             warm_start=False)

In [55]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=8,max_features='sqrt',min_samples_leaf=.1,subsample= .8,random_state=42,n_estimators=50,learning_rate=.2),tuned_parameters,cv=5,scoring='r2')

In [56]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=42, subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [57]:
model.best_score_

-0.004561841780566307

In [58]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=8,max_features='sqrt',min_samples_leaf=.1,subsample= .8,random_state=42,n_estimators=200,learning_rate=.05),tuned_parameters,cv=5,scoring='r2')

In [59]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=42, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [60]:
model.best_score_

-0.005102361695894359

In [61]:
final_model = GradientBoostingRegressor(max_depth=8,max_features='sqrt',min_samples_leaf=.1,subsample=.8,random_state=42,n_estimators=50,learning_rate=.2)

In [62]:
final_model.fit(x,y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=42, subsample=0.8, verbose=0, warm_start=False)

In [63]:
pickle.dump(final_model,open('GBM.pickle','wb'))

In [64]:
y_pred = final_model.predict(x)

In [65]:
r2_score(y,y_pred)

0.0347939822415233